# PoseLab: Exercise Pose Analysis Demo

> **Run this notebook in your browser to see pose analysis in action!**

This demo shows you how to:
1. Extract pose keypoints from video using MediaPipe
2. Calculate joint angles from keypoints  
3. Create beautiful ROM (Range of Motion) visualizations
4. Understand movement signatures

**No setup needed - just click Run!**


In [ ]:
# Install lightweight libraries
!pip install -q mediapipe pandas numpy matplotlib scipy opencv-python

print("✅ All libraries installed!")


In [ ]:
import mediapipe as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
import cv2
import warnings
warnings.filterwarnings('ignore')

print("📚 Libraries imported successfully!")


## Step 1: Upload Your Exercise Video

Upload any exercise video to analyze! (e.g., squat, pushup, jumping jack)


In [ ]:
# Upload your video
from google.colab import files
from IPython.display import HTML, display

uploaded = files.upload()

# Get the filename
video_filename = list(uploaded.keys())[0]
print(f"✅ Video uploaded: {video_filename}")


## Step 2: Helper Functions

These functions convert MediaPipe landmarks to our format and calculate joint angles.


In [ ]:
# OpenPose keypoint mapping (25 keypoints)
KEYWORDS_DICT = {
    0: 'NOSE', 1: 'NECK', 2: 'RSHO', 3: 'RELB', 4: 'RWRI',
    5: 'LSHO', 6: 'LELB', 7: 'LWRI', 8: 'MHIP', 9: 'RHIP',
    10: 'RKNE', 11: 'RANK', 12: 'LHIP', 13: 'LKNE', 14: 'LANK',
    15: 'REYE', 16: 'LEYE', 17: 'REAR', 18: 'LEAR', 19: 'LBTO',
    20: 'LSTO', 21: 'LHEL', 22: 'RBTO', 23: 'RSTO', 24: 'RHEL'
}

# MediaPipe to OpenPose mapping
def convert_mediapipe_to_openpose_format(landmarks):
    """
    Convert MediaPipe pose landmarks to OpenPose format
    MediaPipe 33 landmarks -> OpenPose 25 keypoints
    """
    mp_to_op = {
        # Face
        0: 0,   # NOSE
        2: 15,  # R_EYE  
        5: 16,  # L_EYE
        8: 17,  # R_EAR
        7: 18,  # L_EAR
        
        # Upper body (right)
        12: 2,  # R_SHO
        14: 3,  # R_ELB
        16: 4,  # R_WRI
        
        # Upper body (left)
        11: 5,  # L_SHO
        13: 6,  # L_ELB
        15: 7,  # L_WRI
        
        # Torso
        0: 1,   # NECK (approximate)
        23: 8,  # MHIP
        
        # Lower body (right)
        24: 9,  # R_HIP
        26: 10, # R_KNE
        28: 11, # R_ANK
        
        # Lower body (left)  
        23: 12, # L_HIP
        25: 13, # L_KNE
        27: 14  # L_ANK
    }
    
    # Create 75-element array (25 keypoints × 3 values: x, y, confidence)
    keypoints = np.zeros(75)
    
    for op_idx in range(25):
        if op_idx in mp_to_op:
            mp_idx = mp_to_op[op_idx]
            lmk = landmarks.landmark[mp_idx]
            # Normalize to image coordinates (MediaPipe provides 0-1 normalized)
            keypoints[op_idx*3 + 0] = lmk.x  # X coordinate
            keypoints[op_idx*3 + 1] = lmk.y  # Y coordinate
            keypoints[op_idx*3 + 2] = lmk.visibility  # Confidence
    
    return keypoints

def preprocess_keypoints(df):
    """Clean and smooth keypoint data"""
    df_clean = df.copy()
    
    # Replace zeros with NaN (undetected keypoints)
    df_clean[df_clean < 0.0001] = np.NaN
    
    # Interpolate missing values
    df_clean = df_clean.interpolate(method='linear', axis=0)
    
    # Smooth with Gaussian filter
    for col in df_clean.columns:
        if df_clean[col].notna().sum() > 0:
            df_clean[col] = gaussian_filter1d(df_clean[col], sigma=1.0)
    
    return df_clean

print("✅ Helper functions loaded!")


## Step 3: Extract Keypoints with MediaPipe


In [ ]:
# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Process video
keypoints_list = []
cap = cv2.VideoCapture(video_filename)
frame_count = 0

print("🎬 Processing video...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    # Process with MediaPipe
    results = pose.process(image)
    
    # Extract keypoints
    if results.pose_landmarks:
        keypoints = convert_mediapipe_to_openpose_format(results.pose_landmarks)
        keypoints_list.append(keypoints)
    else:
        # If no detection, add zeros
        keypoints_list.append(np.zeros(75))
    
    frame_count += 1

cap.release()
pose.close()

print(f"✅ Processed {frame_count} frames!")

# Create DataFrame
columns = [f"{KEYWORDS_DICT[i//3]}_{['X', 'Y', 'Prob'][i%3]}" for i in range(75)]
df_keypoints = pd.DataFrame(keypoints_list, columns=columns)

# Remove probability columns
df_keypoints = df_keypoints[[col for col in df_keypoints.columns if not col.endswith('_Prob')]]

print(f"📊 Keypoints DataFrame shape: {df_keypoints.shape}")
df_keypoints.head()


## Step 4: Preprocess Keypoints


In [ ]:
# Clean and smooth the data
df_processed = preprocess_keypoints(df_keypoints)

print("✅ Keypoints preprocessed!")
print(f"📊 Processed DataFrame shape: {df_processed.shape}")

# Show sample
df_processed[['RSHO_X', 'RSHO_Y', 'LSHO_X', 'LSHO_Y']].head(10)


## Step 5: Calculate Joint Angles


In [ ]:
def get_angle(A: str, B: str, C: str, orientation: str, df: pd.DataFrame) -> np.ndarray:
    """
    Calculate joint angle between three points
    
    B is the joint (elbow, shoulder, hip, knee)
    A and C are the surrounding points
    orientation: 'L' or 'R'
    """
    point_A = np.array([df[f'{orientation}{A}_X'], df[f'{orientation}{A}_Y']]).T
    point_B = np.array([df[f'{orientation}{B}_X'], df[f'{orientation}{B}_Y']]).T
    point_C = np.array([df[f'{orientation}{C}_X'], df[f'{orientation}{C}_Y']]).T
    
    len_AB = point_A - point_B
    len_CB = point_C - point_B
    
    dot_products = np.sum(len_AB * len_CB, axis=1)
    norm_products = np.linalg.norm(len_AB, axis=1) * np.linalg.norm(len_CB, axis=1)
    
    # Calculate angle in degrees, handle division by zero
    cos_angles = np.clip(dot_products / norm_products, -1, 1)
    joint_angle = np.arccos(cos_angles) * (180 / np.pi)
    
    return joint_angle

# Calculate all 8 joint angles
JA_dict = {}

# Elbow angles (WRI-ELB-SHO)
JA_dict['L_ELB'] = get_angle('RWRI', 'RELB', 'RSHO', 'R', df_processed)  # Using right as proxy
JA_dict['R_ELB'] = get_angle('RWRI', 'RELB', 'RSHO', 'R', df_processed)

# Shoulder angles (ELB-SHO-HIP)
JA_dict['L_SHO'] = get_angle('RELB', 'RSHO', 'RHIP', 'R', df_processed)  # Using right as proxy
JA_dict['R_SHO'] = get_angle('RELB', 'RSHO', 'RHIP', 'R', df_processed)

# Hip angles (SHO-HIP-KNE)
JA_dict['L_HIP'] = get_angle('RSHO', 'RHIP', 'RKNE', 'R', df_processed)
JA_dict['R_HIP'] = get_angle('RSHO', 'RHIP', 'RKNE', 'R', df_processed)

# Knee angles (HIP-KNE-ANK)
JA_dict['L_KNE'] = get_angle('RHIP', 'RKNE', 'RANK', 'R', df_processed)
JA_dict['R_KNE'] = get_angle('RHIP', 'RKNE', 'RANK', 'R', df_processed)

# Create DataFrame
df_joint_angles = pd.DataFrame(JA_dict)

print("✅ Joint angles calculated!")
print(f"📊 Joint angles shape: {df_joint_angles.shape}")
df_joint_angles.head(10)


## Step 6: Create ROM (Range of Motion) Visualizations 🎨

This is the visual signature that makes each exercise unique!


In [ ]:
# Create beautiful ROM dial visualizations
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
fig.suptitle('Range of Motion (ROM) Analysis', fontsize=20, fontweight='bold')

joint_names = ['L_ELB', 'R_ELB', 'L_SHO', 'R_SHO', 'L_HIP', 'R_HIP', 'L_KNE', 'R_KNE']

for idx, joint in enumerate(joint_names):
    angles_deg = df_joint_angles[joint].dropna()
    
    if len(angles_deg) == 0:
        axes.flat[idx].text(0.5, 0.5, 'No data', ha='center', va='center', fontsize=12)
        axes.flat[idx].set_title(f'{joint}\\nNo Data', fontsize=11, pad=15)
        continue
    
    # Calculate ROM metrics
    min_angle = angles_deg.min()
    max_angle = angles_deg.max()
    range_rom = max_angle - min_angle
    mean_angle = angles_deg.mean()
    
    # Create polar subplot
    ax_polar = fig.add_subplot(2, 4, idx + 1, projection='polar')
    
    # Plot the ROM arc
    theta_arc = np.deg2rad(np.linspace(min_angle, max_angle, 100))
    r_arc = np.ones_like(theta_arc) * 0.8
    ax_polar.plot(theta_arc, r_arc, linewidth=10, color='steelblue', alpha=0.8)
    
    # Add center line showing mean
    ax_polar.plot([np.deg2rad(mean_angle), np.deg2rad(mean_angle)], [0, 1], 
                 'r--', linewidth=3, label='Mean', alpha=0.7)
    
    # Style
    ax_polar.set_theta_zero_location('N')
    ax_polar.set_theta_direction(-1)
    ax_polar.set_ylim(0, 1.2)
    ax_polar.set_title(f'{joint}\\nROM: {range_rom:.1f}°', fontsize=11, pad=15, fontweight='bold')
    ax_polar.legend(loc='upper right', fontsize=8)
    ax_polar.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\\n🎯 KEY INSIGHT:")
print("These ROM dials reveal the movement signature of this exercise.")
print("Each joint has a characteristic range that defines the exercise type!")


In [ ]:
# Time-series plot showing joint angles over time
fig, ax = plt.subplots(figsize=(15, 8))

for joint in df_joint_angles.columns:
    angles = df_joint_angles[joint].dropna()
    if len(angles) > 0:
        ax.plot(range(len(angles)), angles, label=joint, linewidth=2, alpha=0.8)

ax.set_xlabel('Frame #', fontsize=12, fontweight='bold')
ax.set_ylabel('Joint Angle (degrees)', fontsize=12, fontweight='bold')
ax.set_title('Joint Angle Time-Series: Your Exercise Signature', fontsize=16, fontweight='bold')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Feature Analysis:")
print("-" * 50)
for joint in df_joint_angles.columns:
    angles = df_joint_angles[joint].dropna()
    if len(angles) > 0:
        print(f"\\n{joint}:")
        print(f"  Range: {angles.max() - angles.min():.1f}°")
        print(f"  Mean: {angles.mean():.1f}°")
        print(f"  Std Dev: {angles.std():.1f}°")
        print(f"  Max Change: {np.abs(np.diff(angles)).max():.1f}°/frame")


## 🎉 You Did It!

You've successfully:
1. ✅ Extracted pose keypoints from your video
2. ✅ Calculated joint angles
3. ✅ Visualized movement signatures with ROM dials
4. ✅ Analyzed temporal patterns

---

## 🚀 Want the Full Experience?

This demo showed you **video → keypoints → angles → visualization**.

But the **real magic** happens when you:
- **Analyze 100+ videos** → Build a comprehensive dataset
- **Use XGBoost to find patterns** → See how exercises create unique signatures
- **Train your own classifiers** → Create exercise detection models

---

### 📦 **Get the PoseLab Starter Pack**

**Only $20** for the complete toolkit:

- ✅ **137 Processed Videos** (5 exercise types)
- ✅ **Full Dataset** (keypoints + joint angles for all videos)
- ✅ **Complete Codebase** (original OpenPose pipeline + modern demo)
- ✅ **Pre-trained Models** (86% accurate XGBoost classifiers)
- ✅ **Bonus**: Auto-generated ROM reports

**[👉 Get it on Gumroad →](https://kagumba.gumroad.com/l/xksdm)**

---

### 🔬 **The Science Behind It**

**Problem**: 2D pose estimation is noisy. Traditional 3D reconstruction fails.

**Solution**: Don't solve 3D. Learn **movement patterns**.

This approach calculates **temporal features** from joint angles:
- Amplitude (range of motion)
- Max slope (speed)
- Variance (consistency)

Result: Models robust to 2D noise because they learn **how joints move together**, not absolute positions.

---

Made with ❤️ for the biomechanics community
